https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby

In [9]:
import numpy as np
import pandas as pd 

In [24]:
keys = np.array([
        ['A', 'B'],
        ['A', 'B'],
        ['A', 'B'],
        ['A', 'BB'],
        ['C', 'D'],
        ['C', 'D'],
        ['C', 'DD'],
        ['E', 'F'],
        ['E', 'FF'],
        ['G', 'H'] 
        ])

df = pd.DataFrame(
    np.hstack([keys,np.random.randn(10,4).round(2)]), 
    columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
)

df[['col3', 'col4', 'col5', 'col6']] = \
    df[['col3', 'col4', 'col5', 'col6']].astype(float)

In [25]:
df

,col1,col2,col3,col4,col5,col6
0,A,B,-0.64,0.56,0.99,-0.49
1,A,B,-1.16,-0.60,0.33,-0.76
2,A,B,0.22,-0.52,-0.35,0.80
3,A,BB,1.62,-0.59,0.85,0.49
4,C,D,-0.35,-0.14,0.69,0.77
5,C,D,0.76,1.64,0.13,1.05
6,C,DD,-1.65,2.56,-0.22,0.48
7,E,F,-1.11,-0.40,0.15,-1.13
8,E,FF,0.54,0.75,-1.47,-0.04
9,G,H,-0.23,0.01,-0.04,-1.09


In [26]:
df.groupby(['col1', 'col2']).size().reset_index(name='counts')

,col1,col2,counts
0,A,B,3
1,A,BB,1
2,C,D,2
3,C,DD,1
4,E,F,1
5,E,FF,1
6,G,H,1


In [27]:
(df.groupby(['col1', 'col2'])
 .agg({
    'col3': ['mean', 'count'], 
    'col4': ['median', 'min', 'count']
}))

col3         col4            
               mean count median   min count
col1 col2                                   
A    B    -0.526667     3  -0.52 -0.60     3
     BB    1.620000     1  -0.59 -0.59     1
C    D     0.205000     2   0.75 -0.14     2
     DD   -1.650000     1   2.56  2.56     1
E    F    -1.110000     1  -0.40 -0.40     1
     FF    0.540000     1   0.75  0.75     1
G    H    -0.230000     1   0.01  0.01     1

In [28]:
gb = df.groupby(['col1', 'col2'])
counts = gb.size().to_frame(name='counts')
(counts
 .join(gb.agg({'col3': 'mean'}).rename(columns={'col3': 'col3_mean'}))
 .join(gb.agg({'col4': 'median'}).rename(columns={'col4': 'col4_median'}))
 .join(gb.agg({'col4': 'min'}).rename(columns={'col4': 'col4_min'}))
 .reset_index()
)

,col1,col2,counts,col3_mean,col4_median,col4_min
0,A,B,3,-0.526667,-0.52,-0.60
1,A,BB,1,1.620000,-0.59,-0.59
2,C,D,2,0.205000,0.75,-0.14
3,C,DD,1,-1.650000,2.56,2.56
4,E,F,1,-1.110000,-0.40,-0.40
5,E,FF,1,0.540000,0.75,0.75
6,G,H,1,-0.230000,0.01,0.01


In [35]:
df1 = df.groupby('col1').agg({'col2':['min','max'], 'col3':['mean']})
df1.columns = df1.columns.map('_'.join).str.strip('_')
df1

,col2_min,col2_max,col3_mean
col1,,,
A,B,BB,0.010000
C,D,DD,-0.413333
E,F,FF,-0.285000
G,H,H,-0.230000


In [38]:
list(map('|'.join, [('A', 'B'),'BC']))

['A|B', 'B|C']

In [36]:
df[['col1', 'col2']].groupby('col1').agg({'col2':'unique'})

,col2
col1,
A,"[B, BB]"
C,"[D, DD]"
E,"[F, FF]"
G,[H]


In [37]:
# to jsem chtěl
mux = pd.MultiIndex.from_tuples([('A','B'), ('C','D')], 
                                 names=[None, 'date'])
test_df = pd.DataFrame({'col1': 125, 'col2': 225} , index = mux)
test_df

,,col1,col2
,date,,
A,B,125,225
C,D,125,225


In [39]:
test_df.index

MultiIndex([('A', 'B'),
            ('C', 'D')],
           names=[None, 'date'])

In [40]:
test_df.index.values

array([('A', 'B'), ('C', 'D')], dtype=object)

In [43]:
test_df.index.names

FrozenList([None, 'date'])

In [44]:
test_df.values

array([[125, 225],
       [125, 225]])

In [47]:
test_df.groupby('col1').size().reset_index(name='counts')

,col1,counts
0,125,2


In [49]:
test_df.groupby(['col1', 'col2']).size().reset_index(name='counts')

,col1,col2,counts
0,125,225,2
